# This notebook re-trains the last iteration without using Inverse Frequency Weighing for SI Figure S7

In [1]:
from tqdm import tqdm, trange
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import scipy as sp
import time
import sys
import importlib
import os
import hashlib
import subprocess
import gc

from IPython import display
from IPython.display import clear_output
import copy
from copy import deepcopy
from sklearn.metrics import roc_auc_score, roc_curve

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

from captum.attr import IntegratedGradients
from captum.attr import LayerConductance
from captum.attr import NeuronConductance

try:
    import cPickle as pickle
except:
    import pickle
    
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

super_folder="/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4"
pocket_fit_folder=f"{super_folder}/morphing_annealing_4d09_filtered_subset/"
folder=f"{pocket_fit_folder}/prediction_step_5/"
step_db_file=f"{pocket_fit_folder}/ddG_step_4_sigmahole_scaled_dummy_masses/stable_ligs.pickle"
all_ligs_db_file=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_with_2D_no_core_tSNE_and_step5_dG.pickle"
all_no_core_ligs_db_file=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_no_core_ligs_with_step5_dG.pickle"

energy_folder="/home/energy/ykhalak/ML_dG/pde2_dG/generators/structures/set_4"
all_ligs_db_file_energy=f"{energy_folder}/filtered_subset_without_weird_chemistry_with_2D_no_core_tSNE_and_step5_dG.pickle"
all_no_core_ligs_db_file_energy=f"{energy_folder}/filtered_subset_without_weird_chemistry_no_core_ligs_with_step5_dG.pickle"

settings_folder=folder+"/combined_repr_search_with_repeats_and_mparam_scan/settings/"
os.makedirs(settings_folder, exist_ok=True)


Bohr2Ang=0.529177249
RT=0.001985875*300 #kcal/mol

import sys
#sys.path.insert(0, folder)
#sys.path.insert(0, f"{pocket_fit_folder}/prediction_step_0/")
sys.path.append(pocket_fit_folder)
#sys.path.append(folder)
sys.path.append(f"{pocket_fit_folder}/prediction_step_0/")

# Define representation blocks

In [2]:
if 'custom_dataset_modular_with_binning' in sys.modules:
    importlib.reload(sys.modules['custom_dataset_modular_with_binning'])
else:
    import custom_dataset_modular_with_binning
from custom_dataset_modular_with_binning import dataBlocks, CustomMolModularDataset

flags_2D=[dataBlocks.MACCS, dataBlocks.Descriptors, dataBlocks.Graph_desc, dataBlocks.BCUT2D]
flags_3D=[dataBlocks.EState_FP, dataBlocks.Pharmacophore_feature_map,
          dataBlocks.MOE, dataBlocks.MQN, dataBlocks.GETAWAY, dataBlocks.AUTOCORR2D,
          dataBlocks.AUTOCORR3D, dataBlocks.WHIM, dataBlocks.RDF,
          dataBlocks.USR, dataBlocks.USRCUT, dataBlocks.PEOE_VSA, dataBlocks.SMR_VSA,
          dataBlocks.SlogP_VSA, dataBlocks.MORSE]

no_core_flags_2D=[dataBlocks.no_core_MACCS, dataBlocks.no_core_Descriptors, dataBlocks.no_core_Graph_desc, dataBlocks.no_core_BCUT2D]
no_core_flags_3D=[dataBlocks.no_core_EState_FP, dataBlocks.no_core_Pharmacophore_feature_map,
          dataBlocks.no_core_MOE, dataBlocks.no_core_MQN,
          # dataBlocks.no_core_GETAWAY, 
          dataBlocks.no_core_AUTOCORR2D,
          dataBlocks.no_core_AUTOCORR3D, dataBlocks.no_core_WHIM, dataBlocks.no_core_RDF,
          #dataBlocks.no_core_USR, dataBlocks.no_core_USRCUT,
          dataBlocks.no_core_PEOE_VSA, dataBlocks.no_core_SMR_VSA,
          dataBlocks.no_core_SlogP_VSA, dataBlocks.no_core_MORSE]

# import training script

In [3]:
if 'custom_dataset_modular_with_binning' in sys.modules:
    importlib.reload(sys.modules['custom_dataset_modular_with_binning'])
else:
    import custom_dataset_modular_with_binning
from custom_dataset_modular_with_binning import dataBlocks, CustomMolModularDataset

if 'owl_trainer_w_params_general_with_ensemble_summary_model' in sys.modules:
    importlib.reload(sys.modules['owl_trainer_w_params_general_with_ensemble_summary_model'])
else:
    import owl_trainer_w_params_general_with_ensemble_summary_model
from owl_trainer_w_params_general_with_ensemble_summary_model import *

# Scan through represenatation combinations

### full ligs

In [5]:
sfiles=[]
sfiles_step=[]

normalize_x=True
shuffle_seed=12345678

n_Epochs=20000
#hl_w=300
hl_w=20
hl_depth=3
init_learning_rate=5e-3
learning_rate_decay=10000 #order of magnitude in this many epochs
weight_decay=1e-3

normalize_x=True
X_filter=None
impfilt=None

    
weighted=False
use_dropout=True
#shiftY=False
#shiftY=True
shiftY=(1.0, -13.040057410601134)

redo=False

# nsteps=4

# sfiles=[]
# sfiles_step=[]

nrepeats=5

reprs=[]
# names=["2D_3D","MDenerg","atom_hot", "2D_3D_MDenerg_atom_hot"]
# best_hlws=[20,5,50,50]
# best_hlds=[3,3,3,3]
names=["2D_3D"]
best_hlws=[20]
best_hlds=[3]

#2D_3D
representation_flags=[0]*len(dataBlocks)
for b in flags_2D:
    representation_flags[int(b)]=1
for b in flags_3D:
    representation_flags[int(b)]=1
reprs.append(representation_flags)
# #MDenerg
# representation_flags=[0]*len(dataBlocks)
# representation_flags[int(dataBlocks.MDenerg)]=1
# reprs.append(representation_flags)
# #atom_hot
# representation_flags=[0]*len(dataBlocks)
# representation_flags[int(dataBlocks.atom_hot)]=1
# reprs.append(representation_flags)
# # 2D_3D + MDenerg + atom_hot
# representation_flags=[0]*len(dataBlocks)
# for b in flags_2D:
#     representation_flags[int(b)]=1
# for b in flags_3D:
#     representation_flags[int(b)]=1
# representation_flags[int(dataBlocks.MDenerg)]=1
# representation_flags[int(dataBlocks.atom_hot)]=1
# reprs.append(representation_flags)

for ri,representation_flags in enumerate(reprs):
    hl_w=best_hlws[ri]
    hl_depth=best_hlds[ri]
    
    for step in [5]:
        cur_settings_folder=f"{pocket_fit_folder}/prediction_step_{step}/training_w_better_shiftY_v3_no_IFV/settings/"
        os.makedirs(cur_settings_folder, exist_ok=True)


        for rep in range(nrepeats):
            dr_name=f"{names[ri]}_repeat{rep}"

            settings=[
                representation_flags, dr_name, normalize_x, shuffle_seed,
                n_Epochs, hl_w, hl_depth, init_learning_rate, learning_rate_decay, weight_decay,
                impfilt, X_filter,
                weighted, shiftY, use_dropout
                ]

            sha = hashlib.sha256()
            sha.update(pickle.dumps(settings))
            settings_hash=dr_name+"_"+sha.hexdigest()[:10]

            settings_fname=cur_settings_folder+f"{settings_hash}.pickle"
            if(os.path.exists(settings_fname)):
                with open(settings_fname, 'rb') as f:
                    settings_loaded, metrics = pickle.load(f)
                    if(metrics is not None and not redo):
                        print(f"{settings_fname} is already trained.")
                    else:
                        print(f"{settings_fname} exists but is not trained yet.")
    #                     sfiles.append(settings_fname)
    #                     sfiles_step.append(step)
            else:
                pickle.dump( (settings, None), open( settings_fname, "wb" ) )
    #             sfiles.append(settings_fname)
    #             sfiles_step.append(step)

            sfiles.append(settings_fname)
            sfiles_step.append(step)

/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat0_09f1354148.pickle exists but is not trained yet.
/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat1_305590a638.pickle exists but is not trained yet.
/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat2_c1673ad532.pickle exists but is not trained yet.
/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat3_a9ff46c7a6.pickle exists but is not trained yet.
/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_an

In [7]:
print(sfiles)

['/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat0_09f1354148.pickle', '/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat1_305590a638.pickle', '/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat2_c1673ad532.pickle', '/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat3_a9ff46c7a6.pickle', '/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat4_dd2c

# Copy data for the needed ligands

In [106]:
# step=5
# #validation_indeces=[1, 2, 3, 4, 13, 24, 27, 33, 45, 48, 53, 56, 58, 63, 70, 71, 87, 89, 91, 214, 265, 346, 365, 418, 419, 420, 425, 427, 598, 1175, 1230, 1480, 1481, 1741, 1762, 1803, 2148, 2150, 2152, 2292, 2354, 2362, 2416, 2423, 2484, 2498, 3429, 3567, 3569, 3623, 3702, 4028, 5251, 8898, 15508]
# known_ligs_databse_fn=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_with_2D_no_core_tSNE_and_step{step}_dG.pickle"
# with open(known_ligs_databse_fn, 'rb') as f:
#     set4f_ligs = pickle.load(f)
#     known_ligs = [l for l in set4f_ligs if l.HasProp('dG')]
# lig_names = [l.GetProp('ID') for l in known_ligs if l.HasProp('dG')]
# lig_names.append(set4f_ligs[0].GetProp('ID'))
# known_ligs.append(set4f_ligs[0])
# del set4f_ligs
# _=gc.collect()

# validation_db_file=f"{pocket_fit_folder}/expt_validation_ligands.pickle"
# with open(validation_db_file, 'rb') as f:
#     val_ligs = pickle.load(f)
# val_names = [l.GetProp('ID') for l in val_ligs]
# # del val_ligs
# # _=gc.collect()

# lig_names+=val_names
# known_ligs+=val_ligs

In [97]:
# print(val_names)
# print([l.GetProp('dG') for l in val_ligs])

In [108]:
# source_f=f"{pocket_fit_folder}/prediction_step_0/combined_modular_repr_cache"
# destin_f=f"{pocket_fit_folder}/prediction_step_{step}/cached_reprs/combined_modular_repr_cache"

# for representation_flags in tqdm(reprs):
#     repr_hash=hashlib.md5(np.packbits(np.array(representation_flags, dtype=bool)).tobytes()).hexdigest()
# #     print(repr_hash)
    
#     src=f"{source_f}/{repr_hash}"
#     dst=f"{destin_f}/{repr_hash}"
#     os.makedirs(dst, exist_ok=True)
    
#     #for ID in lig_names:
#         #shutil.copy(f"{src}/{ID}.pickle", f"{dst}/{ID}.pickle")
#     for lig in known_ligs:
#         ID=lig.GetProp('ID')
#         with open(f"{src}/{ID}.pickle", 'rb') as f:
#             X,Y = pickle.load(f)
#             Y = np.array([float(lig.GetProp('dG')) if lig.HasProp('dG') else np.nan]) # kcal/mol
#         with open(f"{dst}/{ID}.pickle", 'wb') as f:
#             pickle.dump((X, Y), f)
    
#     shutil.copy(f"{src}/normalization_factors__no_X_filter.dat", f"{dst}/normalization_factors__no_X_filter.dat")

100%|██████████| 4/4 [00:02<00:00,  1.34it/s]


# Run locally

In [ ]:
for i,f in tqdm(enumerate(sfiles)):
    settings_fname=f
    step=sfiles_step[i]
    with open(settings_fname, 'rb') as f:
        settings_loaded, metrics = pickle.load(f)
#         print(i,f, metrics)
        if(metrics is not None):
            continue;
#     if("atom_hot" not in settings_loaded[1]):
#         continue

    print("######################################")
    print(settings_fname)
    print(settings_loaded[1])
    print("######################################")
#     raise()

    train_model_with_settings_general_ensemble(
              settings_fname, False,
              #torch.device("cpu"),
              torch.device("cuda:0"),
              True,
              datafolder=f"{pocket_fit_folder}/prediction_step_5/cached_reprs/",
              #datafolder="/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset/prediction_step_3/combined_repr_search",
              save_folder_base=f"{pocket_fit_folder}/prediction_step_{step}/training_w_better_shiftY_v3_no_IFV",
              all_ligs_db_file=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_with_2D_no_core_tSNE_and_step{step}_dG.pickle",
              all_no_core_ligs_db_file=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_no_core_ligs_with_step{step}_dG.pickle",
              validation_db_file=f"{pocket_fit_folder}/expt_validation_ligands.pickle",
              no_core_validation_db_file=f"{pocket_fit_folder}/expt_validation_ligands_no_core.pickle",
              train_summary_model=True,
              predict_all_ligs=False
             )
#     raise()

print("Done.")

In [110]:
with open("/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset/prediction_step_5/cached_reprs/combined_modular_repr_cache/793f8fc2ef51d558a6cd17022f751801/set4_0.pickle", 'rb') as f:
    temp_lig_repr = pickle.load(f)
print(temp_lig_repr)

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([nan]))


# Copy step 5 databases to energy

In [5]:
import shutil
step=5

all_ligs_db_file=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_with_2D_no_core_tSNE_and_step{step}_dG.pickle"
all_no_core_ligs_db_file=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_no_core_ligs_with_step{step}_dG.pickle"

energy_folder_on_workstation="/netmount/energy/ykhalak/ML_dG/pde2_dG/generators/structures/set_4"
all_ligs_db_file_energy=f"{energy_folder_on_workstation}/filtered_subset_without_weird_chemistry_with_2D_no_core_tSNE_and_step{step}_dG.pickle"
all_no_core_ligs_db_file_energy=f"{energy_folder_on_workstation}/filtered_subset_without_weird_chemistry_no_core_ligs_with_step{step}_dG.pickle"

shutil.copy(all_ligs_db_file, all_ligs_db_file_energy)
shutil.copy(all_no_core_ligs_db_file, all_no_core_ligs_db_file_energy)

'/netmount/energy/ykhalak/ML_dG/pde2_dG/generators/structures/set_4/filtered_subset_without_weird_chemistry_no_core_ligs_with_step5_dG.pickle'

# Run on moa

In [114]:
import queue
import threading

previously_done=0
max_debug_evals=10e8

fits_per_job=1

cmd_str=f"source /etc/profile; module load sge;"
cmd_str_SLURM=f"source /etc/profile;"

fits_written_to_jobs=0
fits_not_trained=0

for step in [5]:
    job_id=0
    fits_this_job=0
    fit_commands_this_job=""
    
    cwd=f"{pocket_fit_folder}/prediction_step_{step}/training_w_better_shiftY_v3/local_runs/"
    os.makedirs(cwd, exist_ok=True)

    save_folder_base=f"{pocket_fit_folder}/prediction_step_{step}/training_w_better_shiftY_v3_no_IFV"
    all_ligs_db_file_energy=f"{energy_folder}/filtered_subset_without_weird_chemistry_with_2D_no_core_tSNE_and_step{step}_dG.pickle"
    all_no_core_ligs_db_file_energy=f"{energy_folder}/filtered_subset_without_weird_chemistry_no_core_ligs_with_step{step}_dG.pickle"
    validation_db_file=f"{pocket_fit_folder}/expt_validation_ligands.pickle"
    no_core_validation_db_file=f"{pocket_fit_folder}/expt_validation_ligands_no_core.pickle"
    
    for i,f in enumerate(tqdm(sfiles)):
        settings_fname=f
    #     step=sfiles_step[i]
        if(not os.path.exists(settings_fname)):
            continue;
        with open(settings_fname, 'rb') as f:
            settings_loaded, metrics = pickle.load(f)
            if(metrics is None and step==sfiles_step[i]):
                fits_not_trained+=1
                fits_this_job+=1
                fit_commands_this_job+=f"""
python /home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset/owl_trainer_w_params_general_with_ensemble_summary_model.py\
 -v -f {settings_fname} --datafolder $TMPDIR --save_folder_base {save_folder_base}\
 --ligsf {all_ligs_db_file_energy} --nocoref {all_no_core_ligs_db_file_energy}\
 --valsf {validation_db_file} --nocorevalsf {no_core_validation_db_file}\
 --no_pred --sm

 """

        if(fits_this_job>=fits_per_job or (i>=len(sfiles)-1 and fits_this_job>0)):

            jobscript_str=f"""#!/bin/bash

#$ -S /bin/bash
#$ -pe openmp_fast 10
#$ -q *
#$ -N pytorch_GPU_worker_{job_id}
#$ -M ykhalak@gwdg.de
#$ -m n
#$ -l h_rt=5:00:00
#$ -wd {cwd}

#SBATCH --job-name=pytorch_SLURM_GPU_worker_{job_id}
#SBATCH --get-user-env
#SBATCH --gres=gpu:1              # number of GPUs requested
#SBATCH --ntasks=1                # Number of MPI process
#SBATCH --cpus-per-task=10        # CPU cores per MPI process
##SBATCH -p p24,p20,p16,p10,p08,p06    # partitions to use
#SBATCH -p p10,p08,p06            # partitions to use
#SBATCH -t 5:00:0                 # hours:min:sec
#SBATCH --chdir={cwd}
#SBATCH -e repr_scan_{job_id}-%j.err
#SBATCH -o repr_scan_{job_id}-%j.out

if [ -n "$NHOSTS" ]; then
    echo "Env. variable NHOSTS found, assuming we are using SGE."
    module load shared                           # access to modules in /cm/shared
else
    echo "Env. variable NHOSTS not set, assuming we are using SLURM."
    export NHOSTS=$SLURM_JOB_NUM_NODES
    export NSLOTS=$SLURM_CPUS_PER_TASK
fi

echo NHOSTS=$NHOSTS
echo NSLOTS=$NSLOTS
echo TMPDIR=$TMPDIR
hostname

echo "#################################################"

export OMP_NUM_THREADS=$NSLOTS
export MKL_NUM_THREADS=$NSLOTS

source ~/.ML_v2_profile
#source ~/.ML_v2_on_energy_profile

cp /home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset/prediction_step_5/cached_reprs/deployable_training_data_with_validation_ligs.tar.gz $TMPDIR/.
cd $TMPDIR
tar -zxf deployable_training_data_with_validation_ligs.tar.gz
cd {cwd}

echo "Done extracting data"

{fit_commands_this_job}
"""

            jobscript_fn=cwd+"/jobscript_{}".format(job_id)
            with open(jobscript_fn,"w") as f:
                f.write(jobscript_str)

            cmd_str+=f" cd {cwd}; qsub {jobscript_fn};"
            cmd_str_SLURM+=f" sbatch {jobscript_fn};"

            job_id+=1
            
            fits_written_to_jobs+=fits_this_job
            fit_commands_this_job=""
            fits_this_job=0
            
            

    print(len(sfiles), fits_not_trained, fits_written_to_jobs)
#     print(fits_this_job, i)
#     raise()

print("Submitting.")
# ssh_cmd_arr=["ssh", "owl", cmd_str]
ssh_cmd_arr=["ssh", "moa2", cmd_str_SLURM]
process = subprocess.Popen(ssh_cmd_arr, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
outs, _ = process.communicate()
process.wait()
print('== subprocess exited with rc =', process.returncode)
print(outs.decode('utf-8'))
print("Done.")

100%|██████████| 20/20 [00:00<00:00, 97.55it/s]


20 10 10
Submitting.
== subprocess exited with rc = 0
Submitted batch job 546053
Submitted batch job 546054
Submitted batch job 546055
Submitted batch job 546056
Submitted batch job 546057
Submitted batch job 546058
Submitted batch job 546059
Submitted batch job 546060
Submitted batch job 546061
Submitted batch job 546062

Done.


# Predict dG locally

In [15]:
for i,f in enumerate(tqdm(sfiles)):
    settings_fname=f
    step=sfiles_step[i]
    with open(settings_fname, 'rb') as f:
        settings_loaded, metrics = pickle.load(f)
#         print(i,sfiles[i], metrics.summary_model_best_pred)
        if(metrics is None):
            continue;
        elif(metrics.summary_model_best_pred is not None):
            continue;

    print("######################################")
    print(settings_fname, "step", step)
    print("######################################")
#     raise()

    Predict_from_saved_models_ensemble(settings_fname, redo=True,
                                      #torch.device("cpu"),
                                      device=torch.device("cuda:0"),
                                      show_progress=True,
                                      #datafolder="/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset/prediction_step_3/combined_repr_search",
                                      datafolder="/home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset/prediction_step_0",
                                      save_folder_base=f"{pocket_fit_folder}/prediction_step_{step}/training_w_better_shiftY_v3_no_IFV",
                                      all_ligs_db_file=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_with_2D_no_core_tSNE_and_step{step}_dG.pickle",
                                      all_no_core_ligs_db_file=f"{pocket_fit_folder}/filtered_subset_without_weird_chemistry_no_core_ligs_with_step{step}_dG.pickle",
                                      max_cache_Mem_MB=1024*6,
                                     )
#     raise()

print("Done.")

100%|██████████| 5/5 [00:00<00:00, 26.03it/s]

0 /home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat0_09f1354148.pickle (array([-12.298728, -11.937565,  -9.948077, ..., -11.572161, -11.708302,
       -11.591374], dtype=float32), array([0.19254494, 0.19440372, 0.2765522 , ..., 0.26204905, 0.4239824 ,
       0.50241536], dtype=float32))
1 /home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/settings/2D_3D_repeat1_305590a638.pickle (array([-12.042234, -11.967661, -10.505789, ..., -11.039019, -11.164343,
       -12.223458], dtype=float32), array([0.27306846, 0.08075845, 0.19139366, ..., 0.23397279, 0.28487682,
       0.68114614], dtype=float32))
2 /home/ykhalak/Projects/ML_dG/pde2_dG/generators/structures/set_4/morphing_annealing_4d09_filtered_subset//prediction_step_5/training_w_better_shiftY_v3_no_IFV/s